In [3]:
import os
import json
import openai
import random
import pandas as pd
import numpy as np
from manifest import Manifest
from sklearn.metrics import f1_score
from collections import defaultdict
from datasets import load_dataset

## Set-up Manifest

In [4]:
manifest_openai = Manifest(
        client_name="openai",
        cache_name="sqlite", 
        client_connection="text-davinci-002",
        cache_connection="./decomposition_manifest.log",
        stop_token="---",
        temperature=0.0,
        max_tokens=3,
        top_p=1.0,
        n=1,
    )

In [ ]:
# include path to
DATASET_FOLDER = os.environ['DATASET_PATH']
DATA_PATH = os.path.join(DATASET_FOLDER, "data_transformation")

## Evaluate

### Dataset: benchmark-stackoverflow

In [13]:
accuracy = []
transform_split = "benchmark-stackoverflow"
base_dir = f"{DATA_PATH}{transform_split}"
for x in os.listdir(base_dir):
    task = x.split(".txt")[0]
    file = open(os.path.join(base_dir, x), 'r')
    lines = file.readlines()
    instruction = lines[0].split("//")[-1].strip("\n")
    instruction = f"{instruction}."
    if "txt" in x.split(".")[-1]: 
        file = pd.read_csv(os.path.join(base_dir, x), sep="\t\t",  encoding='cp1252', 
                           names=["input", "output"], index_col = False, skiprows=1, engine='python')   
        icl = ""
        for x, row in file[0:3].iterrows():
            icl += f"input: {row.input}\noutput: {row.output}\n\n"
        sub_acc = []
        for x, row in file[3:].iterrows():
            input = f"{instruction}\n\n{icl}input: {row.input}\noutput:"
            gpt_out = manifest_openai.run(input, max_tokens=30, stop_token="\n", temperature=0.0)
            row_out = str(row.output).lower().strip()
            gpt_out = str(gpt_out).lower().strip()
            if row_out == gpt_out: sub_acc.append(1)
            else: sub_acc.append(0)
        if len(sub_acc) == sum(sub_acc): 
            # count as correct only if model gets all samples per transformation correct
            accuracy.append(1)
        else: 
            accuracy.append(0)


print(f"Accuracy on benchmark-stackoverflow: {sum(accuracy) / len(accuracy)}")

Accuracy on benchmark-stackoverflow: 0.6530612244897959


### Dataset: benchmark-BinqQuery (semantic)

In [18]:
accuracy = []
transform_split = "benchmark-bing-query-logs"
base_dir = f"{DATA_PATH}{transform_split}"
for x in os.listdir(base_dir):
    task = x.split(".txt")[0]
    file1 = open(os.path.join(base_dir, x), 'r')
    lines = file1.readlines()
    instruction = lines[0].strip("\n")
    if "txt" in x.split(".")[-1] and "semantic" in x: 
        file = pd.read_csv(os.path.join(base_dir, x), sep="\t\t",  encoding='cp1252', 
                           names=["input", "output"], index_col = False, skiprows=1, engine='python')
        icl = "" 
        for x, row in file[0:3].iterrows():
            icl += f"input: {row.input}\noutput: {row.output}\n\n"
        sub_acc = []
        for x, row in file[3:].iterrows():
            input = f"{instruction}\n\n{icl}input: {row.input}\noutput:"
            gpt_out = manifest_openai.run(input, max_tokens=30, stop_token="\n", temperature=0.0)
            row_out = str(row.output).lower().strip()
            gpt_out = str(gpt_out).lower().strip()
            if row_out == gpt_out: sub_acc.append(1)
            else: sub_acc.append(0)
        if len(sub_acc) == sum(sub_acc): 
            accuracy.append(1)
        else:
            accuracy.append(0)
print(f"Accuracy on benchmark-bing-query-logs: {sum(accuracy) / len(accuracy)}")

Accuracy on benchmark-bing-query-logs: 0.54
